# 06　(没)　神ぼ大νττ　CTGAN

### メモ  
GAOZが近いのを選ぶ、後藤Electionでも無理だった

Random Forestによる識別りつが0.38と、低すぎはしないので、Arfの攻撃をかける

### 担当者と日付
2024/10/3 渡邉

## 準備

### ライブラリのインポート  

In [1]:
import pandas as pd
import numpy as np
import warnings #ヘッダとかのやつが邪魔だったのでファイルにまとめてインポートしてる
warnings.simplefilter(action='ignore', category=FutureWarning)
!pip install sdv
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer
from sdv.sampling import Condition


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# fmt: off
ALL_HEADER = ['Name', 'Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '247', '260', '653', '673', '810', '885',  # noqa: E501
                      '1009', '1073', '1097', '1126', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017',  # noqa: E501
                      '2021', '2043', '2086', '2087', '2093', '2100', '2105', '2138', '2143', '2174', '2193', '2253',  # noqa: E501
                      '2399', '2628', '2797', '2872', '2968', '3393', '3438', '3439', '3440', '3466', '3479', '3489',  # noqa: E501
                      '3877', '3889'] # noqa

B_HEADERS_LIST = [
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '653', '1525', '2105', '2193', '2253', '2628', '2872', '3438', '3439', '3440', '3877', '3889'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '260', '653', '673', '1009', '1073', '1525', '1750', '1881', '1967', '2043', '2093', '2105', '2143', '2193', '2399', '2628', '2968', '3479', '3489', '3877', '3889'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '673', '1881', '1920', '2087', '2138'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '673', '810', '885', '1009', '1073', '1097', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2043', '2087', '2093', '2138', '2399', '3438', '3439', '3440'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '673', '810', '1073', '1126', '1702', '2100', '2174', '2253', '2797', '3393', '3466'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '885', '1097', '1654', '2086', '2138', '2872'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '2100', '2143', '2872', '3479'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '1097', '1750', '2021', '2093', '2105', '2628', '2968'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '1920', '2017', '2087'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '1097', '2628', '2174', '2797', '1073', '2100', '2968', '2105', '2193'],  # noqa: E501
]

B_USER_ATTRIBUTE_HEADERS = ['Gender', 'Age', 'Occupation', 'ZIP-code']

B_REVIEW_HEADERS_LIST = [
  ['260', '653', '1525', '2105', '2193', '2253', '2628', '2872', '3438', '3439', '3440', '3877', '3889'],  # noqa: E501
  ['2', '56', '260', '653', '673', '1009', '1073', '1525', '1750', '1881', '1967', '2043', '2093', '2105', '2143', '2193', '2399', '2628', '2968', '3479', '3489', '3877', '3889'],  # noqa: E501
  ['673', '1881', '1920', '2087', '2138'],  # noqa: E501
  ['2', '56', '673', '810', '885', '1009', '1073', '1097', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2043', '2087', '2093', '2138', '2399', '3438', '3439', '3440'],  # noqa: E501
  ['673', '810', '1073', '1126', '1702', '2100', '2174', '2253', '2797', '3393', '3466'],  # noqa: E501
  ['247', '885', '1097', '1654', '2086', '2138', '2872'],  # noqa: E501
  ['247', '2100', '2143', '2872', '3479'],  # noqa: E501
  ['260', '1097', '1750', '2021', '2093', '2105', '2628', '2968'],  # noqa: E501
  ['247', '1920', '2017', '2087'],  # noqa: E501
  ['260', '1097', '2628', '2174', '2797', '1073', '2100', '2968', '2105', '2193'],  # noqa: E501
]

MOVIE_IDS = ['2', '56', '247', '260', '653', '673', '810', '885', '1009', '1073', '1097', '1126', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2021', '2043', '2086', '2087', '2093', '2100', '2105', '2138', '2143', '2174', '2193', '2253', '2399', '2628', '2797', '2872', '2968', '3393', '3438', '3439', '3440', '3466', '3479', '3489', '3877', '3889']
# fmt: on

### C_0 ~ C_9までの読み込み

In [3]:
from os import listdir

ATTACK_TARGET = "06"

#c0~c9に対するdfのリスト
c_data_list = []

files = listdir("../data")

for i in range(10):
    file = None
    for f in files:
        if f.startswith(f"C{ATTACK_TARGET}_{i}"):
            file = f
            break
    if file is None:
        print(f)
        raise("File not found")
    c_data = pd.read_csv(f"../data/C{ATTACK_TARGET}_{i}.csv")
    c_data_list.append(c_data)

c_data_list[0]

,Gender,Age,Occupation,ZIP-code,260,653,1525,2105,2193,2253,2628,2872,3438,3439,3440,3877,3889
0,M,1,8,48,0,5,3,3,5,4,3,2,0,0,5,3,0
1,M,1,15,959,4,3,0,2,1,1,1,0,3,1,4,1,0
2,F,50,19,114,0,2,0,4,3,0,3,3,0,0,0,3,0
3,F,35,1,433,0,2,0,3,4,4,0,0,4,0,0,0,2
4,M,35,14,370,2,5,0,0,4,5,2,0,1,2,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,F,56,6,193,3,3,0,1,1,5,1,2,4,0,0,0,1
9996,F,1,14,939,2,0,0,0,2,1,5,0,1,4,0,5,3
9997,M,35,2,662,1,4,5,3,4,3,2,1,5,0,2,0,3
9998,F,25,19,590,3,5,5,0,0,0,1,0,3,1,0,0,3


### Ba, Bbの読み込み

In [4]:
Ba = pd.read_csv(f"../data/B{ATTACK_TARGET}a.csv", names=ALL_HEADER)
Ba = Ba.drop(0)
Ba.head()

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
1,Andriana Lickorish,F,25,3,850,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nananne Willerstone,F,50,20,980,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Calla Jesse,F,35,3,931,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Avrom Skarr,M,45,18,778,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Benedick Cresswell,M,50,13,446,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
Bb = pd.read_csv(f"../data/B{ATTACK_TARGET}b.csv")
Bb.head()

,2,56,247,260,653,673,810,885,1009,1073,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,3,0,0,2,2,0,0,0,0,*,...,0,3,4,0,0,0,5,5,0,0
1,2,0,1,4,3,0,4,0,0,0,...,4,1,1,4,0,0,0,1,5,0
2,2,0,5,5,5,0,4,0,0,5,...,4,5,0,0,0,0,4,4,0,0
3,1,4,2,4,0,4,0,0,*,5,...,3,2,3,1,4,2,1,5,2,2
4,2,1,0,4,1,5,1,2,4,0,...,1,4,4,0,0,0,5,4,0,0


## C3のモデルを作成

In [6]:
c_index = 3

metadata = SingleTableMetadata()

for header in B_HEADERS_LIST[c_index]:
    metadata.add_column(header, sdtype="categorical")

model = CTGANSynthesizer(
    metadata, verbose=True, enforce_min_max_values=True, epochs=1000
)

model.fit(c_data_list[c_index])

/Users/ken.watanabe/Work/Research/PWS/pws2024_attack/venv/lib/python3.11/site-packages/sdv/single_table/base.py:105: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


Gen. (-4.00) | Discrim. (-0.06): 100%|██████████| 1000/1000 [21:23<00:00,  1.28s/it]


In [7]:
fig = model.get_loss_values_plot()
fig.show()

### Cのモデルで。Cの実際の値を入れたら値が近くなるか確認する

In [31]:
generated_df = model.sample(40000)

In [35]:
# added_df = model.sample(1000000)
# generated_df = pd.concat([generated_df, added_df])

# 条件付きサンプリング (Genderが'F', Ageが25のデータを生成)
# gaoz = "M,18,16,453"
gaoz = "F,25,3,850"
# gaoz = "F,50,18,498"
# gaoz = "M,25,11,501"
# gaoz = "M,25,7,803"
gaoz="F,50,20,980"
gaoz="F,35,3,931"
gaoz="M,18,16,453"

gender, age, occupation, zipcode = gaoz.split(",")
age = int(age)
occupation = int(occupation)
zipcode = int(zipcode)

# condition = Condition(
#     {"Gender": gender, "Age": age, "Occupation": occupation, "ZIP-code": zipcode},
#     num_rows=20,
# )
# print(condition)
# conditional_synthetic_data = model.sample_from_conditions(
#     [condition],
# )

matched_df = generated_df[
    (generated_df["Gender"] == gender) &
    (generated_df["Age"] == age) &
    (generated_df["Occupation"] == occupation) &
    (generated_df["ZIP-code"] == zipcode)
]
matched_df

,Gender,Age,Occupation,ZIP-code,2,56,673,810,885,1009,...,1967,2017,2043,2087,2093,2138,2399,3438,3439,3440
11344,M,18,16,453,0,0,5,0,1,1,...,0,5,2,3,0,0,1,2,0,4
89897,M,18,16,453,5,1,5,4,1,1,...,4,3,2,5,3,0,5,1,1,5
101090,M,18,16,453,4,1,3,4,1,1,...,4,0,1,3,3,0,0,2,2,3
105157,M,18,16,453,2,1,5,1,0,0,...,2,5,1,4,2,0,5,1,4,0
107776,M,18,16,453,0,0,1,4,4,1,...,0,3,0,5,0,3,0,4,3,0
110446,M,18,16,453,2,0,0,1,0,2,...,0,0,2,2,0,0,5,0,2,0
113534,M,18,16,453,0,0,2,2,3,0,...,2,0,2,2,0,0,1,4,4,2
133149,M,18,16,453,2,1,0,4,1,1,...,0,0,0,2,3,4,1,1,2,0
166949,M,18,16,453,0,0,2,1,1,0,...,0,2,2,3,0,3,0,0,3,0
169379,M,18,16,453,0,0,1,1,1,0,...,2,0,0,4,2,0,1,4,3,3


In [36]:
from sdv.evaluation import evaluate

df = c_data_list[c_index]

score = evaluate(
    real_data=df[df['Gender'] == 'M'],  # 条件に合う実データをフィルタリング
    synthetic_data=matched_df,
    metadata=metadata
)
print(f"評価スコア: {score}")

ImportError: cannot import name 'evaluate' from 'sdv.evaluation' (/Users/ken.watanabe/Work/Research/PWS/pws2024_attack/venv/lib/python3.11/site-packages/sdv/evaluation/__init__.py)

## C0~C9のためのモデルの作成

In [10]:
from arfpy import arf

c_models = []

for i in range(10):
    c_train_data = c_data_list[i].astype('category')
    c_model = arf.arf(x = c_train_data, num_trees=200, early_stop=False)
    c_model.forde()
    c_models.append(c_model)

Initial accuracy is 0.52295
Iteration number 1 reached accuracy of 0.39305.
Initial accuracy is 0.539
Iteration number 1 reached accuracy of 0.42805.
Initial accuracy is 0.55185
Iteration number 1 reached accuracy of 0.3468.
Initial accuracy is 0.61125
Iteration number 1 reached accuracy of 0.44175.
Initial accuracy is 0.60025
Iteration number 1 reached accuracy of 0.4021.
Initial accuracy is 0.53785
Iteration number 1 reached accuracy of 0.3567.
Initial accuracy is 0.5581
Iteration number 1 reached accuracy of 0.34745.
Initial accuracy is 0.5522
Iteration number 1 reached accuracy of 0.36955.
Initial accuracy is 0.56035
Iteration number 1 reached accuracy of 0.35285.
Initial accuracy is 0.52925
Iteration number 1 reached accuracy of 0.38445.


In [11]:
import pickle

pickle.dump(c_models, open(f"../models/{ATTACK_TARGET}_gan_arf.pkl", 'wb'))

In [12]:
c_train_data = c_data_list[3].astype('category')
c_model = arf.arf(x = c_train_data, num_trees=200)
c_model.forde()
c_models[3] = c_model

Initial accuracy is 0.61545
Iteration number 1 reached accuracy of 0.43955.


In [13]:
c_data_list[3].head()

,Gender,Age,Occupation,ZIP-code,2,56,673,810,885,1009,...,1967,2017,2043,2087,2093,2138,2399,3438,3439,3440
0,F,1,16,730,0,4,2,3,0,4,...,0,2,1,0,5,2,5,2,3,0
1,M,56,3,245,0,0,0,0,2,0,...,0,0,5,0,0,0,4,3,5,0
2,F,35,14,544,1,0,4,2,4,3,...,0,5,2,2,1,4,3,3,5,2
3,F,18,10,857,0,4,0,0,0,0,...,0,0,4,5,0,0,5,3,0,3
4,F,25,0,12,0,2,0,0,5,2,...,5,0,2,0,2,4,5,2,0,0


In [14]:
generated_df = c_models[3].forge(90000)

matched_df = generated_df[
    (generated_df["Gender"] == "F") &
    (generated_df["Age"] == 1) &
    (generated_df["Occupation"] == 16) &
    (generated_df["ZIP-code"] == 730)
]
matched_df

,Gender,Age,Occupation,ZIP-code,2,56,673,810,885,1009,...,1967,2017,2043,2087,2093,2138,2399,3438,3439,3440


In [15]:
# added_df = c_models[3].forge(30000)
# generated_df = pd.concat([generated_df, added_df])

gaoz = "M,56,16,730"
gaoz = "F,35,14,544"

gender = gaoz.split(",")[0]
age = int(gaoz.split(",")[1])
occupation = int(gaoz.split(",")[2])
zipcode = int(gaoz.split(",")[3])

matched_df = generated_df[
    (generated_df["Gender"] == gender) &
    (generated_df["Age"] == age) &
    (generated_df["Occupation"] == occupation) &
    (generated_df["ZIP-code"] == zipcode)
]
matched_df

,Gender,Age,Occupation,ZIP-code,2,56,673,810,885,1009,...,1967,2017,2043,2087,2093,2138,2399,3438,3439,3440


In [16]:
def get_matched_data(generated_data, gender, age, occupation, zipcode):
    matched_data = generated_data[
        (generated_data["Gender"] == gender) &
        (generated_data["Age"] == age) &
        (generated_data["Occupation"] == occupation) &
        (generated_data["ZIP-code"] == zipcode)
    ]
    return matched_data

row_index = 1
c_index = 3

generated_data = c_models[c_index].forge(30000)

# for i in range(10):
gender = Ba.iloc[row_index]["Gender"]
age = int(Ba.iloc[row_index]["Age"])
occupation = int(Ba.iloc[row_index]["Occupation"])
zipcode = int(Ba.iloc[row_index]["ZIP-code"])

print(f"{gender}, {age}, {occupation}, {zipcode}")

matched_data = get_matched_data(generated_data, gender, age, occupation, zipcode)

while len(matched_data) == 0 and len(generated_data) < 100000:
    added_data = c_models[c_index].forge(10000)
    generated_data = pd.concat([generated_data, added_data])
    print("Added data")
    matched_data = get_matched_data(generated_data, gender, age, occupation, zipcode)

matched_data

F, 50, 20, 980
Added data
Added data
Added data
Added data
Added data
Added data
Added data


,Gender,Age,Occupation,ZIP-code,2,56,673,810,885,1009,...,1967,2017,2043,2087,2093,2138,2399,3438,3439,3440


In [17]:
def get_matched_data(generated_data, gender, age, occupation, zipcode):
    matched_data = generated_data[
        (generated_data["Gender"] == gender) &
        (generated_data["Age"] == age) &
        (generated_data["Occupation"] == occupation) &
        (generated_data["ZIP-code"] == zipcode)
    ]
    return matched_data

row_index = 8
c_index = 3

# for i in range(10):
gender = Ba.iloc[row_index]["Gender"]
age = int(Ba.iloc[row_index]["Age"])
occupation = int(Ba.iloc[row_index]["Occupation"])
zipcode = int(Ba.iloc[row_index]["ZIP-code"])

print(f"{gender}, {age}, {occupation}, {zipcode}")

matched_data = get_matched_data(generated_data, gender, age, occupation, zipcode)

while len(matched_data) == 0 and len(generated_data) < 200000:
    added_data = c_models[c_index].forge(10000)
    generated_data = pd.concat([generated_data, added_data])
    print("Added data")
    matched_data = get_matched_data(generated_data, gender, age, occupation, zipcode)

matched_data

F, 1, 19, 786
Added data
Added data
Added data
Added data
Added data


,Gender,Age,Occupation,ZIP-code,2,56,673,810,885,1009,...,1967,2017,2043,2087,2093,2138,2399,3438,3439,3440
1967,F,1,19,786,1,0,1,0,4,4,...,0,0,0,2,0,1,0,0,0,3


### GAOZに関して、BaのN番目の行と完全一致する行をC0~C9から取得　　
-GAOZへの変更が行われているか

In [18]:
# 試しにBaの3行目を選択
N = 3

gender = Ba.iloc[N]["Gender"]
age = int(Ba.iloc[N]["Age"])
occupation = int(Ba.iloc[N]["Occupation"])
zipcode = int(Ba.iloc[N]["ZIP-code"])

print("検索条件")
print(gender, age, occupation, zipcode)

検索条件
M 45 18 778


### GAOZが完全一致するものでなく、各Ciの中でHamming distが最小のものをリスト化する

In [19]:
import copy

def GAOZ_weighted_hamming(comp, i_gender, i_age, i_occupation, i_zipcode):
    ham = 0
    if comp['Gender'] != i_gender:
        ham += 1
    if comp['Age'] != i_age:
        ham += 2
    if  comp['Occupation'] != i_occupation:
        ham += 2
    if comp['ZIP-code'] != i_zipcode:
        ham += 4
    return ham

filtered_C_list = []
for c_index, cf in enumerate(c_data_list):
    min_hamming_distance = float("inf")
    cand = []
    # Cの10000レコードのそれぞれにハミング距離を計算
    for index, c_row in cf.iterrows():
        hamming_distance = GAOZ_weighted_hamming(c_row, gender, age, occupation, zipcode)
        if hamming_distance == min_hamming_distance:
            cand.append((c_row, hamming_distance))
        if hamming_distance < min_hamming_distance:
            min_hamming_distance = hamming_distance
            cand = [(c_row, hamming_distance)]
    # ハミング距離とCのファイルのindexを追加
    for elem, hamming_distance in cand:
        headers = copy.deepcopy(ALL_HEADER)
        headers.append("GAOZ_Hamming_Dist")
        headers.append("C_index")
        elem["GAOZ_Hamming_Dist"] = hamming_distance
        elem["C_index"] = c_index
        filtered_C_list.append(pd.Series(elem, index=headers))

merged_df = pd.DataFrame(filtered_C_list)
merged_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,3438,3439,3440,3466,3479,3489,3877,3889,GAOZ_Hamming_Dist,C_index
7246,NaN,F,45,14,778,NaN,NaN,NaN,0.0,2.0,...,2.0,0.0,0.0,NaN,NaN,NaN,0.0,2.0,3,0
9382,NaN,F,45,15,778,NaN,NaN,NaN,5.0,4.0,...,3.0,0.0,0.0,NaN,NaN,NaN,3.0,2.0,3,0
5756,NaN,M,45,0,778,1.0,3.0,NaN,1.0,0.0,...,NaN,NaN,NaN,NaN,0.0,1.0,0.0,5.0,2,1
801,NaN,F,45,10,778,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2
5920,NaN,F,45,15,778,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9030,NaN,M,45,18,62,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,8
9490,NaN,M,25,14,778,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,8
9603,NaN,M,45,18,435,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,8
3779,NaN,F,18,18,778,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,9


### データの再現　　
各行に対して最頻値を一つ値を取得する

In [20]:
import statistics
from collections import Counter

def select_random_mode(column):
    """
    最頻値を取得
    最頻値が複数ある場合はランダムに1つ選択
    """
    # NaNを除外した値のリストを作成
    valid_values = column.dropna().tolist()
    if not valid_values:
        # 有効な値がない場合はNaNを返す
        return np.nan
    # 最頻値とその出現回数を取得
    value_counts = Counter(valid_values)
    max_count = max(value_counts.values())
    # 最頻値のリストを作成
    modes = [value for value, count in value_counts.items() if count == max_count]
    # 最頻値が1つの場合はそれを返し、複数ある場合はランダムに1つ選択
    return np.random.choice(modes)

def select_random_value(column):
    # NaNを除外した値のリストを作成
    valid_values = column.dropna().tolist()
    if valid_values:
        # 有効な値がある場合、ランダムに1つ選択
        return np.random.choice(valid_values)
    else:
        # 有効な値がない場合はNaNを返す
        return np.nan

# 各カラムからランダムに値を選択
# random_values = {}
# for column in ALL_HEADER:
#     random_values[column] = select_random_value(merged_df[column])

# # 結果を新しいデータフレームとして作成
# result_df = pd.DataFrame([random_values])
# result_df

## 攻撃

### 以上の処理をBaの各行（0~49行）に対して実行する

In [21]:
import pandas as pd
import numpy as np

final_df = pd.DataFrame(columns=ALL_HEADER)

for row_index in range(50):
    # Baデータフレームから現在の行の情報を取得
    gender = Ba.iloc[row_index]["Gender"]
    age = int(Ba.iloc[row_index]["Age"])
    occupation = int(Ba.iloc[row_index]["Occupation"])
    zipcode = int(Ba.iloc[row_index]["ZIP-code"])

    filtered_C_list = []
    for c_index, cf in enumerate(c_data_list):
        min_hamming_distance = float("inf")
        cand = []
        # Cの10000レコードのそれぞれにハミング距離を計算
        for index, c_row in cf.iterrows():
            hamming_distance = GAOZ_weighted_hamming(c_row, gender, age, occupation, zipcode)
            if hamming_distance == min_hamming_distance:
                cand.append((c_row, hamming_distance))
            if hamming_distance < min_hamming_distance:
                min_hamming_distance = hamming_distance
                cand = [(c_row, hamming_distance)]
        # ハミング距離とCのファイルのindexを追加
        for elem, hamming_distance in cand:
            headers = copy.deepcopy(ALL_HEADER)
            headers.append("GAOZ_Hamming_Dist")
            headers.append("C_index")
            elem["GAOZ_Hamming_Dist"] = hamming_distance
            elem["C_index"] = c_index
            filtered_C_list.append(pd.Series(elem, index=headers))

    merged_df = pd.DataFrame(filtered_C_list)

    candidate_rows_df = pd.DataFrame(filtered_C_list)

    # 各列に対して、ランダムに値を選択（多数決の実装）
    elected_value = {}
    for column in ALL_HEADER:
        elected_value[column] = select_random_value(candidate_rows_df[column])

    # 選択された値から新しい行を作成
    elected_value_row_df = pd.DataFrame([elected_value])

    # 選択された行をfinal_dfの最後に追加
    final_df = pd.concat([final_df, elected_value_row_df], ignore_index=True)

final_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,NaN,F,25,8,850,0.0,0.0,2.0,1.0,0.0,...,4.0,5.0,4.0,0.0,0.0,3.0,0.0,5.0,0.0,0.0
1,NaN,F,50,18,980,5.0,0.0,4.0,5.0,5.0,...,3.0,5.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,4.0
2,NaN,M,56,9,931,3.0,2.0,4.0,2.0,0.0,...,0.0,1.0,5.0,0.0,0.0,1.0,0.0,1.0,2.0,2.0
3,NaN,M,45,18,946,1.0,3.0,2.0,0.0,2.0,...,1.0,4.0,2.0,5.0,0.0,0.0,0.0,1.0,0.0,2.0
4,NaN,M,50,6,446,5.0,0.0,0.0,3.0,0.0,...,5.0,5.0,5.0,1.0,2.0,0.0,0.0,0.0,2.0,3.0
5,NaN,F,45,20,975,2.0,2.0,0.0,3.0,0.0,...,1.0,5.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0
6,NaN,M,25,20,501,4.0,0.0,0.0,0.0,1.0,...,2.0,3.0,0.0,0.0,0.0,1.0,4.0,3.0,0.0,0.0
7,NaN,M,18,10,919,3.0,2.0,2.0,4.0,0.0,...,0.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,3.0,5.0
8,NaN,F,1,0,786,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,4.0,0.0,4.0,5.0,2.0,0.0,2.0,0.0
9,NaN,M,56,1,88,0.0,2.0,2.0,5.0,0.0,...,1.0,4.0,5.0,0.0,0.0,0.0,5.0,4.0,3.0,0.0


### 最終的に得た映画のレビューの候補とBbを比較し、Bbの何番目のindexが一番近くてハミング距離にどれくらいの違いがあるかを取得する

In [22]:

def hamming_distance(a, b):
    a_int = a.fillna(0).astype(int)
    b_int = b.replace('*', None).fillna(0).astype(int)
    # ハミング距離の計算
    return np.sum(a_int != b_int)

def find_closest_row(row, Bb):
    distances = Bb.apply(lambda bb_row: hamming_distance(row[MOVIE_IDS], bb_row[MOVIE_IDS]), axis=1)
    min_distance = distances.min()
    closest_index = distances.idxmin()
    return pd.Series({'Min_Hamming_Distance': min_distance, 'Closest_Bb_Index': closest_index})

# 結果を格納するデータフレーム
result_df = pd.DataFrame(index=final_df.index, columns=['Min_Hamming_Distance', 'Closest_Bb_Index'])

# 各行に対してハミング距離を計算し、最も近い行を見つける
for idx, row in final_df[MOVIE_IDS].iterrows():
    result = find_closest_row(row, Bb[MOVIE_IDS])
    result_df.loc[idx] = result

# 元のfinal_dfにハミング距離の結果を結合
final_result_df = pd.concat([final_df, result_df], axis=1)
final_result_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,3438,3439,3440,3466,3479,3489,3877,3889,Min_Hamming_Distance,Closest_Bb_Index
0,NaN,F,25,8,850,0.0,0.0,2.0,1.0,0.0,...,4.0,0.0,0.0,3.0,0.0,5.0,0.0,0.0,20,22
1,NaN,F,50,18,980,5.0,0.0,4.0,5.0,5.0,...,4.0,2.0,0.0,0.0,0.0,0.0,0.0,4.0,25,30
2,NaN,M,56,9,931,3.0,2.0,4.0,2.0,0.0,...,5.0,0.0,0.0,1.0,0.0,1.0,2.0,2.0,28,33
3,NaN,M,45,18,946,1.0,3.0,2.0,0.0,2.0,...,2.0,5.0,0.0,0.0,0.0,1.0,0.0,2.0,24,43
4,NaN,M,50,6,446,5.0,0.0,0.0,3.0,0.0,...,5.0,1.0,2.0,0.0,0.0,0.0,2.0,3.0,27,40
5,NaN,F,45,20,975,2.0,2.0,0.0,3.0,0.0,...,3.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,23,4
6,NaN,M,25,20,501,4.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,4.0,3.0,0.0,0.0,24,36
7,NaN,M,18,10,919,3.0,2.0,2.0,4.0,0.0,...,2.0,0.0,0.0,3.0,0.0,0.0,3.0,5.0,28,43
8,NaN,F,1,0,786,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,4.0,5.0,2.0,0.0,2.0,0.0,22,22
9,NaN,M,56,1,88,0.0,2.0,2.0,5.0,0.0,...,5.0,0.0,0.0,0.0,5.0,4.0,3.0,0.0,28,4
